# Clear Previous Files

In [1]:
# Clear previous files
!rm -rf ./Rounds
!rm -rf ./Client_data

# Model description

In [2]:
NUM_CLIENTS = 2 # Only used to split the data set

# Data Set creation

In [3]:
import tensorflow as tf
import numpy as np

# Download MNIST dataset
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()

# Check shapes
print("Training data shape:", train_X.shape, train_y.shape)
print("Test data shape:", test_X.shape, test_y.shape)

2024-12-17 20:37:35.082014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 20:37:35.108597: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 20:37:35.108626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 20:37:35.108657: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 20:37:35.115271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 20:37:35.115858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Training data shape: (60000, 28, 28) (60000,)
Test data shape: (10000, 28, 28) (10000,)


In [4]:
clients_data = []
split_factor = 1
for i in range(NUM_CLIENTS):
    select = len(train_X) // split_factor
    shuffle_idx = np.random.permutation(train_X.shape[0])
    shuffled_x = train_X[shuffle_idx]
    shuffled_y = train_y[shuffle_idx]
    train_X_selected = shuffled_x[:select]
    train_y_selected = shuffled_y[:select]
    clients_data.append((train_X_selected, train_y_selected))
print(clients_data[0][0].shape)
import os
os.mkdir('./Client_data')
for i, (client_X, client_y) in enumerate(clients_data):
    print(f"Client {i+1}: X shape = {client_X.shape}, y shape = {client_y.shape}")
    np.savez(f"./Client_data/client_{i+1}_data.npz",
             X=client_X, y=client_y)
    print(f"Saved as file client_{i+1}_data.npz")

(60000, 28, 28)
Client 1: X shape = (60000, 28, 28), y shape = (60000,)
Saved as file client_1_data.npz
Client 2: X shape = (60000, 28, 28), y shape = (60000,)
Saved as file client_2_data.npz


# Creating the model

In [5]:
import tensorflow as tf
import tensorflow_federated as tff
import os  # future use
import sys # future use
from contextlib import redirect_stdout # future use 
tf.get_logger().setLevel('ERROR') # Supress tensor flow messages

# Define the model
def generate_model():
        model = tf.keras.Sequential([
                tf.keras.layers.Flatten(input_shape=(28, 28)),
                tf.keras.layers.InputLayer(input_shape=(784,)),
                tf.keras.layers.Dense(10, kernel_initializer='zeros'),
                tf.keras.layers.Softmax(),
        ])
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9), 
                        loss='sparse_categorical_crossentropy', 
                        metrics=['accuracy'])
        return model

server_model = generate_model()

server_model.save('./Rounds/0/server_model_round_0')

2024-12-17 20:37:37.875898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-17 20:37:37.876169: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
generate_model().summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 input_2 (InputLayer)        multiple                  0         
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
 softmax_1 (Softmax)         (None, 10)                0         
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Clients train the model

In [7]:
def client(i: int, e: int, r: int):
    """
    i -> client number
    e -> no of epoches
    r -> current server round
    """
    
    # Load Data
    
    data = np.load(f'./Client_data/client_{i}_data.npz')
    X = data['X']
    y = data['y']

    # Load Server Model

    model = tf.keras.models.load_model(f'./Rounds/{r-1}/server_model_round_{r-1}')

    # Train the model

    history = model.fit(X, y, epochs=e, verbose=0)

    log(i, r, history)

    # Save the model
    with open(os.devnull, 'w') as f, redirect_stdout(f):
        model.save(f'./Rounds/{r}/client_models/client_{i}_round_{r}_model')

In [8]:
def fetch_client_models(r, e):    
    for i in range(1, NUM_CLIENTS+1):
        client(i, e, r)

# Server updates the main model

In [9]:
def update_server_model(round):

  import tensorflow_federated as tff

  # Load all client models

  client_models = [tf.keras.models.load_model(f'./Rounds/{round}/client_models/client_{i}_round_{round}_model') for i in range(1, NUM_CLIENTS+1)]

  client_weights = [model.get_weights() for model in client_models]

  def average_weights(weights):
      """
      Returns the average of a given list of weights.

      Args:
        weights: A list of weights, where each element is a list of weights for a single layer.

      Returns:
        A list of averaged weights.
      """
      return [(sum(w) / len(w)) for w in zip(*weights)]

  average_weights = average_weights(client_weights)

  # Save the new model

  server_model = generate_model()

  

  server_model.set_weights(average_weights)
  with open(os.devnull, 'w') as f, redirect_stdout(f):
    server_model.save(f'./Rounds/{round}/server_model_round_{round}') 

  print("Server model created and saved successfully!")

# Trend of server model developemnt

In [10]:

def evaluation():
    global MAX_ROUNDS
    for round in range(0, MAX_ROUNDS+1):
        server_model = tf.keras.models.load_model(f'./Rounds/{round}/server_model_round_{round}')
        loss, accuracy = server_model.evaluate(test_X, test_y, verbose=1) # verbose=1 prints progress bar

        print(f"Test Loss: {loss:.4f}")
        print(f"Test Accuracy: {accuracy*100:.4f}%")

# Generate Logs for client & server model performance

In [11]:
def log(i, r, history):
    global LOG
    if r not in LOG:
        LOG[r] = {}
    LOG[r][i] = history

def print_log(r):
    print(f"Round {r}")
    for i in range(1, NUM_CLIENTS+1):
        print(f"Client {i}")
        if i in LOG[r]:  # Ensure client data exists
            print(f"Loss : {LOG[r][i].history['loss'][-1]}")
            print(f"Accuracy : {LOG[r][i].history['accuracy'][-1] * 100:.2f}%")
        else:
            print("No data logged.")

    print("Server")
    server_model = tf.keras.models.load_model(f'./Rounds/{r}/server_model_round_{r}')
    loss, accuracy = server_model.evaluate(test_X, test_y, verbose=1) # verbose=1 prints progress bar

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy*100:.4f}%")


# Orchestration

In [12]:
# no of epochs 
e = 5
# Run for MAX_ROUNDS

LOG = {}

MAX_ROUNDS = 10
for r in range(1, MAX_ROUNDS+1):
    fetch_client_models(r, e)
    update_server_model(r)
    print_log(r)
evaluation()

Server model created and saved successfully!
Round 1
Client 1
Loss : 3919.89697265625
Accuracy : 9.92%
Client 2
Loss : 3889.63671875
Accuracy : 9.93%
Server
313/313 [==============================] - 0s 766us/step - loss: 4071.3169 - accuracy: 0.1026
Test Loss: 4071.3169
Test Accuracy: 10.2600%
Server model created and saved successfully!
Round 2
Client 1
Loss : 4011.587646484375
Accuracy : 9.91%
Client 2
Loss : 4032.720458984375
Accuracy : 9.92%
Server
313/313 [==============================] - 0s 863us/step - loss: 3288.3191 - accuracy: 0.1082
Test Loss: 3288.3191
Test Accuracy: 10.8200%
Server model created and saved successfully!
Round 3
Client 1
Loss : 3924.025634765625
Accuracy : 9.93%
Client 2
Loss : 3807.34765625
Accuracy : 9.93%
Server
313/313 [==============================] - 0s 748us/step - loss: 3288.3569 - accuracy: 0.0942
Test Loss: 3288.3569
Test Accuracy: 9.4200%
Server model created and saved successfully!
Round 4
Client 1
Loss : 3854.637939453125
Accuracy : 9.91%
Cli